In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
dataset = pd.read_csv('data/board_games.csv')

print(dataset.dtypes)
print(dataset.describe())

# Handling missing values (example: fill with mean or remove)
dataset.fillna(dataset.mean(numeric_only=True), inplace=True)  # or dataset.dropna(inplace=True)

# Encoding categorical variables if necessary
label_encoders = {}
for column in dataset.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    dataset[column] = label_encoders[column].fit_transform(dataset[column])

# Feature Scaling
scaler = StandardScaler()
scaled_features = scaler.fit_transform(dataset.drop('users_rated', axis=1))

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(scaled_features, dataset['users_rated'], test_size=0.2, random_state=42)

# Initialize the regressors
lgbm = LGBMRegressor()
xgb = XGBRegressor()
catboost = CatBoostRegressor(verbose=0)  # 'verbose=0' to prevent a lot of output

# Fit the models
lgbm.fit(X_train, y_train)
xgb.fit(X_train, y_train)
catboost.fit(X_train, y_train)

# Make predictions
predictions_lgbm = lgbm.predict(X_test)
predictions_xgb = xgb.predict(X_test)
predictions_catboost = catboost.predict(X_test)

# Evaluate the models
mae_lgbm = mean_absolute_error(y_test, predictions_lgbm)
mse_lgbm = mean_squared_error(y_test, predictions_lgbm)
r2_lgbm = r2_score(y_test, predictions_lgbm)

mae_xgb = mean_absolute_error(y_test, predictions_xgb)
mse_xgb = mean_squared_error(y_test, predictions_xgb)
r2_xgb = r2_score(y_test, predictions_xgb)

mae_catboost = mean_absolute_error(y_test, predictions_catboost)
mse_catboost = mean_squared_error(y_test, predictions_catboost)
r2_catboost = r2_score(y_test, predictions_catboost)

print('\n\n')
print('LightGBM:')
print(f'MAE: {mae_lgbm}')
print(f'MSE: {mse_lgbm}')
print(f'R-squared: {r2_lgbm}')
print('------')
print('XGBoost:')
print(f'MAE: {mae_xgb}')
print(f'MSE: {mse_xgb}')
print(f'R-squared: {r2_xgb}')
print('------')
print('CatBoost:')
print(f'MAE: {mae_catboost}')
print(f'MSE: {mse_catboost}')
print(f'R-squared: {r2_catboost}')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001337 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3567
[LightGBM] [Info] Number of data points in the train set: 8425, number of used features: 21
[LightGBM] [Info] Start training from score 854.795252



LightGBM:
MAE: 820.4596557661026
MSE: 5077811.930874889
R-squared: 0.39957234076976567
------
XGBoost:
MAE: 879.1472577776001
MSE: 5412649.853329113
R-squared: 0.35997931276132866
------
CatBoost:
MAE: 822.7151470124151
MSE: 5043447.190606735
R-squared: 0.4036358115796638
